In [27]:
import datetime as dt
from urllib.request import urlopen
import bs4
import pandas as pd

In [28]:
def date_format(d):
    d = str(d).replace('-', '.')
    
    yyyy = int(d.split('.')[0]) 
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])

    this_date= dt.date(yyyy, mm, dd)
    return this_date

In [29]:
def historical_index_naver(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    #index_cd = 'KOSPI' 'KOSDAQ' 'KPI200''FUT' 
    
    if start_date:   # start_date가 있으면
        start_date = date_format(start_date)   # date 포맷으로 변환
    else:    # 없으면
        start_date = dt.date.today()   # 오늘 날짜를 지정
    if end_date:   
        end_date = date_format(end_date)   
    else:   
        end_date = dt.date.today()  
        
        
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_index).read()         # 지정한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
    
    dates = source.find_all('td', class_='date')        # <td class="date">태그에서 날짜 수집   
    prices = source.find_all('td', class_='number_1')   # <td class="number_1">태그에서 지수 수집
    
    # 아래에서 this_data는 page_n = 1에서 나온 date를 사용하기 때문에 오늘 날짜 포함 6개의 이전 날자들을 포함(최신)
    # 이 후로 historical_index_naver함수가 page_n값을 증가(다음 페이지) 시키면서 점점 날짜(this_date)는 과거로 가게 되고
    # this_data이 start_date을 지나 더 과거로 가게 되는 순간 지금까지 저장된 dic값을 리턴하고 종료
    
    # 특정 시점이후 데이터가 없는 경우(선물의 경우)가 발생하면 naver_index에 의해 urlopen이 fail이 나거나
    # 문제가 없더라도 source내에는 우리가 원하는 data가 들어 있지 않을 것이다. 그런 상태에서 dates와 prices를
    # 추출하는 것이 의미 없기는 하겠지만 일단 추출한다 해도 dates의 값은 Null로 td나 class관련해서 아무 것도 없게 될 것이고
    # 그것은 len(dates)가 0이 되어 딕션너리에 아무런 저장을 하지 않겠지만 page_n == last_page가 될 때까지 
    # 쓸데 없이 반복 호출 되는 비효율적인면을 개선할 필요가 있다. (위쪽에 체크 루틴을 넣어 주면 좋을 듯)
    for n in range(len(dates)):
    
        if dates[n].text.split('.')[0].isdigit():
            
            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:   
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리
                this_close = prices[n*4].text   # prices 중 종가지수인 0,4,8,...번째 데이터 추출
                this_close = this_close.replace(',', '')
                this_close = float(this_close)

                # 딕셔너리에 저장
                historical_prices[this_date] = this_close
                
            elif this_date < start_date:   
            # start_date 이전이면 함수 종료 (설정한 시작점 보다 더 과거의 data는 더 이상 처리 하지 않는다)
                return historical_prices              
            
    # 페이지 네비게이션 : 매번 재귀호출 때마다 반복 실행 시킬 필요 없을 것 같은데.. 구조 잘 못 짠것 같음
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
        last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
        last_page = int(last_page)   # 숫자형 변수로 변환
        
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)   
    
    # 이것이 호출 된다는 것은 page_n == last_page 인 경우이다.
    return historical_prices  

In [30]:
def historical_global_daum(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    #index_cd = 'KOSPI' 'KOSDAQ' 'KPI200''FUT' 
    if start_date:   # start_date가 있으면
        start_date = date_format(start_date)   # date 포맷으로 변환
    else:    # 없으면
        start_date = dt.date.today()   # 오늘 날짜를 지정
    if end_date:  
        end_date = date_format(end_date)   
    else:   
        end_date = dt.date.today()  
    
    url = 'http://finance.daum.net/global/index_daily.daum?type=default&ric=/.' + index_cd + '&page=' + str(page_n)

    source = urlopen(url).read()
    source = bs4.BeautifulSoup(source, 'lxml')

    dates = source.find_all('td', class_='datetime')   # <td class="datetime">태그에서 날짜 수집
    prices = source.find_all('td', class_='num')   # <td class="num">태그에서 날짜 수집

    rows_in_page = len(dates)

    # 데이터가 없는 경우는 바로 dic값 가지고 리턴
    # 일반적으로 10일 것이고 마지막 페이지만 행이 10 이하일 가능성이 높다.
    if len(dates) > 0:

        for n in range(rows_in_page):

            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)

            if this_date <= end_date and this_date >= start_date:   
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리, 0, 3, 6, 9와 같은 순서로 종가를 가진 index이며 실제 내용에 종가 외에
                # 천단위 외에 다양한 \t, \n과 같은 데이터들이 있으므로 그것들을 지워준다.
                this_close = prices[n*3].text
                this_close = this_close.replace(' ', '')
                this_close = this_close.replace('\t', '')
                this_close = this_close.replace('\n', '')
                this_close = this_close.replace(',', '')
                this_close = float(this_close)

                # 딕셔너리에 저장
                historical_prices[this_date] = this_close
                
            elif this_date < start_date:   
            # start_date 이전이면 함수 종료
                return historical_prices                         
        
        # 페이지 네비게이션
        # 다음은 페이지당 행이 10개가 기본임 그러므로 10 인 경우는 다음번 페이지로 진행이 가능함.
        # 10이 아닌 경우는 마지막 페이지일 것이므로 return한다.
        if rows_in_page == 10:
            page_n = int(page_n)
            page_n = page_n + 1
            
            historical_global_daum(index_cd, start_date, end_date, page_n, last_page)
            
    return historical_prices  

In [31]:
#index_cd = 'KOSPI' 'KOSDAQ' 'KPI200''FUT' 
historical_prices = dict()
#historical_index_naver(index_cd, '2018-1-1', '2018-7-1')
my_own_dic = historical_index_naver('KOSPI', '2018-7-1', '2018-8-1')
my_own_dic

{datetime.date(2018, 8, 1): 2307.07,
 datetime.date(2018, 7, 31): 2295.26,
 datetime.date(2018, 7, 30): 2293.51,
 datetime.date(2018, 7, 27): 2294.99,
 datetime.date(2018, 7, 26): 2289.06,
 datetime.date(2018, 7, 25): 2273.03,
 datetime.date(2018, 7, 24): 2280.2,
 datetime.date(2018, 7, 23): 2269.31,
 datetime.date(2018, 7, 20): 2289.19,
 datetime.date(2018, 7, 19): 2282.29,
 datetime.date(2018, 7, 18): 2290.11,
 datetime.date(2018, 7, 17): 2297.92,
 datetime.date(2018, 7, 16): 2301.99,
 datetime.date(2018, 7, 13): 2310.9,
 datetime.date(2018, 7, 12): 2285.06,
 datetime.date(2018, 7, 11): 2280.62,
 datetime.date(2018, 7, 10): 2294.16,
 datetime.date(2018, 7, 9): 2285.8,
 datetime.date(2018, 7, 6): 2272.87,
 datetime.date(2018, 7, 5): 2257.55,
 datetime.date(2018, 7, 4): 2265.46,
 datetime.date(2018, 7, 3): 2272.76,
 datetime.date(2018, 7, 2): 2271.54}

In [41]:
#index_cd = 'GSPC' = S&P500 'IXIC'=NASDAQ 'DJI'= DOW
#index_cd = 'N225' = 니케이225 'SSEC'=중국상해종합 'HSI'=홍콩항셍
historical_prices = dict()
#historical_index_naver(index_cd, '2018-1-1', '2018-7-1')
sp500 = historical_global_daum('SSEC', '2018-1-1', '2018-8-8')
sp500

{datetime.date(2018, 8, 8): 2744.07,
 datetime.date(2018, 8, 7): 2779.37,
 datetime.date(2018, 8, 6): 2705.16,
 datetime.date(2018, 8, 3): 2740.44,
 datetime.date(2018, 8, 2): 2768.02,
 datetime.date(2018, 8, 1): 2824.53,
 datetime.date(2018, 7, 31): 2876.4,
 datetime.date(2018, 7, 30): 2869.05,
 datetime.date(2018, 7, 27): 2873.59,
 datetime.date(2018, 7, 26): 2882.23,
 datetime.date(2018, 7, 25): 2903.65,
 datetime.date(2018, 7, 24): 2905.56,
 datetime.date(2018, 7, 23): 2859.54,
 datetime.date(2018, 7, 20): 2829.27,
 datetime.date(2018, 7, 19): 2772.55,
 datetime.date(2018, 7, 18): 2787.26,
 datetime.date(2018, 7, 17): 2798.13,
 datetime.date(2018, 7, 16): 2814.04,
 datetime.date(2018, 7, 13): 2831.18,
 datetime.date(2018, 7, 12): 2837.66,
 datetime.date(2018, 7, 11): 2777.77,
 datetime.date(2018, 7, 10): 2827.63,
 datetime.date(2018, 7, 9): 2815.11,
 datetime.date(2018, 7, 6): 2747.23,
 datetime.date(2018, 7, 5): 2733.88,
 datetime.date(2018, 7, 4): 2759.13,
 datetime.date(2018, 7,